In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from config import api_key
from scipy.stats import linregress
from pprint import pprint

In [2]:
artist = 'FooFighters'
locale = 'en-us'
segment = 'Music'

events = requests.get(f'https://app.ticketmaster.com/discovery/v2/events.json?keyword={artist}&segment={segment}&locale={locale}&apikey={api_key}').json()


In [3]:
open('test_output.json', 'w').write(json.dumps(events, indent=4))

805072

In [4]:
for x in events['_embedded']['events']:
   # print(x.keys())
   # print(x['_embedded']['venues'][0]['city']['name'])
    artist_name = x['name']
    event_date = x['dates']['start']['localDate']
    event_city = x['_embedded']['venues'][0]['city']['name']
    event_state = x['_embedded']['venues'][0]['state']['stateCode']
    min_price = x['priceRanges'][0]['min']
    max_price = x['priceRanges'][0]['max']
    lat = x['_embedded']['venues'][0]['location']['latitude']
    long =x['_embedded']['venues'][0]['location']['longitude']
    


KeyError: 'priceRanges'

In [ ]:
# Parse out event data
#event = events['_embedded']['events']

artist_name = event['name']
event_date = event['dates']['start']['localDate']
event_city = event['_embedded']['venues'][0]['city']['name']
event_state = event['_embedded']['venues'][0]['state']['stateCode']
min_price = event['priceRanges'][0]['min']
max_price = event['priceRanges'][0]['max']
lat = event['_embedded']['venues'][0]['location']['latitude']
long = event['_embedded']['venues'][0]['location']['longitude']

ff_events_df = pd.DataFrame({'Artist': [artist_name], 'Date': [event_date], 'City': [event_city], 'State': [event_state], 'Min Price': [min_price], 'Max Price': [max_price], 'Lat': [lat], 'Lon': [long]})

print(ff_events_df)